In [1]:
import math

import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm.notebook import tqdm
import sys

import ddpm
import datasets

import os
import trimesh

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

print(f"PyTorch version Installed: {torch.__version__}\nTorchvision version Installed: {torchvision.__version__}\n")
if not torch.__version__.startswith("1.11"):
    print("you are using an another version of PyTorch. We expect PyTorch 1.11.0. You may continue using your version but it"
          " might cause dependency and compatibility issues.")
if not torchvision.__version__.startswith("0.12"):
    print("you are using an another version of torchvision. We expect torchvision 0.12. You can continue with your version but it"
          " might cause dependency and compatibility issues.")

PyTorch version Installed: 2.0.1+cu117
Torchvision version Installed: 0.15.2+cu117

you are using an another version of PyTorch. We expect PyTorch 1.11.0. You may continue using your version but it might cause dependency and compatibility issues.
you are using an another version of torchvision. We expect torchvision 0.12. You can continue with your version but it might cause dependency and compatibility issues.


In [3]:
# DATA NORMALIZATION FUNCTION PRIOR TO DATASET GENERATION

# Import dataset class
from grasp_object_dataset import graspDataset
from torch.utils.data import DataLoader

# Function to obtain mean, std, max, and min of given dataset
def get_mean_std(main_dir, object_dir, dataset):

    # Allocate variables
    full_joints = []

    # Load all samples - only joint values!
    for sample in tqdm(dataset):
        joints = sample[0]
        full_joints.append(joints)

    mean = np.mean(full_joints, axis = 0)
    std = np.std(full_joints, axis = 0)
    max = np.max(full_joints, axis = 0)
    min = np.min(full_joints, axis = 0)
    

    return mean, std, max, min

In [5]:
# DATASET AND DATALOADER DEFINITION

# Import dataset class
from grasp_object_dataset import graspDataset
from torch.utils.data import DataLoader

# Directories of dataset grasps + objects: set to a small version of it to overfit!! Full version found in dataset_XX_full
main_dir = './dataset_grasps_full/'
object_dir = './dataset_objects_full/'

# Generate dataset with all dataset samples
main_dataset = graspDataset(main_dir, object_dir, mode = 'train', split = {'train': 1, 'val': 0, 'test': 0}, normalization=None, transform_joint = None, transform_object = None)

# Calculate mean, std, max, min of full dataset
mean_std_max_min = list(get_mean_std(main_dir, object_dir, main_dataset))
# Print not norm data
print('NOT NORMALIZED: ', mean_std_max_min)

# Datasets definition
train_dataset = graspDataset(main_dir, object_dir, mode = 'train', split = {'train': 1, 'val': 0, 'test': 0}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)
#val_dataset = graspDataset(main_dir, object_dir, mode = 'val', split = {'train': 1, 'val': 0, 'test': 0}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)
#test_dataset = graspDataset(main_dir, object_dir, mode = 'test', split = {'train': 1, 'val': 0, 'test': 0}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)

# Comparison for normalized dataset
mean_std_max_min_2 = list(get_mean_std(main_dir, object_dir, train_dataset))
print('NORMALIZED: ', mean_std_max_min_2)

# Dataloader definition
train_dataloader = DataLoader(train_dataset , batch_size=64, shuffle=True, num_workers=2, drop_last=False)
#val_dataloader = DataLoader(val_dataset , batch_size=64, shuffle=True, num_workers=2, drop_last=False)
#test_dataloader = DataLoader(test_dataset , batch_size=64, shuffle=True, num_workers=2, drop_last=False)

  0%|          | 0/21000 [00:00<?, ?it/s]

NOT NORMALIZED:  [array([ 3.98561844e-02,  2.57192661e-01,  5.95373810e-01,  1.67236000e-01,
        2.65023496e-02,  2.65105132e-01,  6.80795671e-01,  1.57267439e-01,
        1.17773209e-02,  3.03083575e-01,  6.61583456e-01,  1.70557048e-01,
        2.61156612e-01, -1.07516990e-01,  2.03640379e-01,  5.48817102e-01,
        2.04976352e-01,  2.73408066e-01,  1.04242190e+00, -1.19935514e-02,
       -2.36260475e-01, -6.06011368e-02, -6.05525100e-02,  8.50713967e-03,
       -2.46173353e-02,  6.38794029e-05, -3.58334739e-03, -7.04945023e-04]), array([0.08841094, 0.12425186, 0.13485115, 0.17028817, 0.08242855,
       0.10374933, 0.19098119, 0.17153302, 0.09209367, 0.11868084,
       0.17730865, 0.17665884, 0.10540849, 0.08868023, 0.09023743,
       0.23112383, 0.19124254, 0.22412766, 0.11790531, 0.10479109,
       0.20813474, 0.08463916, 1.78267916, 0.65887648, 1.78051084,
       0.09024645, 0.09977398, 0.08307887]), array([0.44093332, 0.72629118, 1.05269575, 0.85050714, 0.34213725,
       0

  0%|          | 0/21000 [00:00<?, ?it/s]

NORMALIZED:  [array([-0.12520227, -0.05825086, -0.03482967, -0.56303924, -0.00694221,
       -0.09080227,  0.1099189 , -0.58760699, -0.0257935 , -0.01083718,
       -0.02048508, -0.49064162,  0.0814    , -0.11457864, -0.10987953,
       -0.16722104, -0.49478828, -0.01662538,  0.27163743, -0.02297611,
       -0.02433884,  0.43323216, -0.01928046,  0.00724428, -0.00791501,
        0.00340821, -0.00509778, -0.00069022]), array([0.24803257, 0.28030283, 0.30514164, 0.38954828, 0.26296456,
       0.29795695, 0.32953832, 0.40987787, 0.24830535, 0.30361581,
       0.30488528, 0.36616864, 0.24984337, 0.2441268 , 0.27511499,
       0.33594732, 0.40974628, 0.38852221, 0.25021091, 0.36027187,
       0.37334508, 0.27221381, 0.5674609 , 0.42084961, 0.56687281,
       0.36866746, 0.40530449, 0.34397149]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.

In [4]:
import os
import numpy as np
from sklearn.decomposition import PCA

num_components = 10
pca = PCA(n_components=num_components)

# Iterate through batches
for batch in train_dataloader:
    joint_angles_batch = batch[0]
    # Fit the PCA model to the batch and transform it to reduced dimensionality
    reduced_data = pca.fit_transform(joint_angles_batch)
    # Inverse transform to reconstruct the batch
    reconstructed_data = pca.inverse_transform(reduced_data)

    # Check the shape of the reconstructed batch
    print("Shape of reconstructed batch:", reconstructed_data.shape)

ModuleNotFoundError: No module named 'sklearn'